In [ ]:
import numpy as np
from abc import ABC, abstractmethod
from objective_function import *
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
class objective_func(ABC):
    @abstractmethod
    def func(self, x):
        pass
    @abstractmethod
    def dfunc(self, x):
        pass
    @abstractmethod
    def get_optimal(self):
        pass
    @abstractmethod
    def get_optimum(self):
        pass
    def visualise1d(self, lim, n):
        ''' 
            lim: the visualisation scope [-lim, lim] in each dimension
            n: the number of points used to interpolate between [-lim, lim]
        '''
        xs = np.linspace(-lim, lim, n)
        fs = []
        for x in xs:
            fs.append(self.func(x))
        ax = plt.plot(xs, fs)
        return ax
    def visualise2d(self, lim, n):
        x, y = np.linspace(-lim, lim, n), np.linspace(-lim, lim, n)
        xx, yy = np.meshgrid(x, y)
        zz = np.zeros(xx.shape)
        for j in range(n):
            for i in range(n):
                zz[j, i] = self.func(np.array([x[i], y[j]]))
        fig = plt.figure(figsize=(7,7))
        ax = fig.add_subplot(111)
        sc = ax.scatter(x=xx.ravel(), y=yy.ravel(), c=zz.ravel(), cmap='YlGnBu')
        ax.set_title("function value f(x1, x2)")
        ax.set_xlabel("x1 axis")
        ax.set_ylabel("x2 axis")
        ax.scatter(x=[self.optimal[0]], y=[self.optimal[1]], c='red', marker='x')
        plt.colorbar(sc)
        fig.show()
        return ax
    def visualise3d(self, lim, n):
        x, y = np.linspace(-lim, lim, n), np.linspace(-lim, lim, n)
        z = []
        for i in y:
            z_line = []
            for j in x:
                z_line.append(self.func(np.array([j,i])))
            z.append(z_line)
        fig = go.Figure(data=[go.Surface(z=z, x=x, y=y),  \
                              go.Scatter3d(x=[self.optimal[0]], y=[self.optimal[1]], z=[self.optimum])])
        fig.update_layout(autosize=False,
                          scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                          width=500, height=500,
                          margin=dict(l=65, r=50, b=65, t=90))
        fig.show()
    def visulise_gradient(self, lim, n):
        x, y = np.linspace(-lim, lim, n), np.linspace(-lim, lim, n)
        xx, yy = np.meshgrid(x, y)
        zz = np.zeros((n, n, 2))
        for j in range(len(y)):
            for i in range(len(x)):
                zz[j, i, :] = self.dfunc(np.array([x[i], y[j]]))
        fig = plt.figure(figsize=(8,8))
        ax = fig.add_subplot(111)
        ax.quiver(xx,yy,zz[:,:,0],zz[:,:,1])
        ax.scatter(x=[self.optimal[0]], y=[self.optimal[1]], c='red', marker='x')
        fig.show()
        return ax
    def visualise2d_section(self, pos, dire):
        ''' 
            pos: the position of cross-section
            dire: along this direction/dimension to get cross-section
        '''
        fig = plt.figure(figsize=(4,4))
        xs = np.linspace(-self.lim, self.lim, 301)
        fs = []
        if dire == 'x':
            for x in xs:
                fs.append(self.func(np.array([x, pos])))
        else:
            for x in xs:
                fs.append(self.func(np.array([pos, x])))
        ax = plt.plot(xs, fs)
        if(isinstance(self, (ackley, tuned_ackley))):
            plt.ylim((0,25))
            plt.xlim((-25,25))
        fig.show()
        return ax
    def visualize2d_section_gradient(self, pos, dire):
        fig = plt.figure(figsize=(4,4))
        xs = np.linspace(-self.lim, self.lim, 300)
        dfs = []
        if dire == 'x':
            for x in xs:
                dfs.append(self.dfunc(np.array([x, pos])))
        else:
            for x in xs:
                dfs.append(self.dfunc(np.array([pos, x])))
        dfs = np.array(dfs)
        plt.plot(xs, dfs[:,0])
        plt.plot(xs, dfs[:,1])
        fig.show()

### Ackley function
\begin{equation*}
f(x) = -20 \exp \left( -\frac{1}{5}\sqrt{ \frac{1}{d} \sum_{i=1}^{d} x_i^2 }\right) -
\exp\left( \frac{1}{d} \sum_{i=1}^{d} \cos\left(2\pi x_i \right) \right) + 20 + e, \\
f'(x) = \left(... , g\left(x_j\right), ...\right)^d, \\
g(x_j) = 4\frac{x_j}{d\sqrt{ \frac{1}{2} \sum_{i=1}^{2} x_i^2 } } 
\exp \left( -\frac{1}{5}\sqrt{ \frac{1}{d} \sum_{i=1}^{d} x_i^2 }\right) +
\frac{2\pi}{d} \sin(2\pi x_j) \exp\left( \frac{1}{d} \sum_{i=1}^{d} \cos\left(2\pi x_i \right) \right),\\
f^*(x^*)=0, x^*=(0)^d
\end{equation*}

In [ ]:
class ackley(objective_func):
    def __init__(self, dim=2):
        self.optimum = 0
        self.lim = 25
        self.dim = dim
        self.optimal = np.zeros((self.dim, ))
    def func(self, x):
        '''
        the period of local minimum along each axis is 1, integer coordinate (1,1), (2,3)... 
        x and y is interchangeable
        global minimum is 0 with arguments x=y=0
        local minimums far away from orgin are 20
        supremum is 20 + e - 1/e = 22.35
        symmetric along x=0, y=0, y=x lines
        disappearing global gradient when far away from optimal
        '''
        arg1 = -0.2 * np.sqrt(np.power(x, 2).mean())
        arg2 = np.cos(2*np.pi*x).mean()
        return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e
    def dfunc(self, x):
        if np.linalg.norm(x) == 0:
            return x
        arg1 = -0.2 * np.sqrt(np.power(x, 2).mean())
        arg2 = np.cos(2*np.pi*x).mean()
        g = lambda xx: -0.8 * xx / arg1 * np.exp(arg1) / self.dim + 2 * np.pi * np.sin(2 * np.pi * xx) * np.exp(arg2) / self.dim
        return g(x)
    def get_optimal(self):
        return self.optimal
    def get_optimum(self):
        return self.optimum
a = ackley()
print("check func value: expected:", a.get_optimum(), ", actual:", a.func(a.get_optimal()))
print("check gradient: expected:[0,0], actual:",a.dfunc(a.get_optimal()))

In [ ]:
a.visualise1d(lim=15, n=300)

In [ ]:
a.visualise2d_section(50.0, 'x')

In [ ]:
a.visualise2d_section(25, 'x')

In [ ]:
a.visualise2d_section(0, 'x')
plt.title("original ackley, f(x1,0) cross section")
plt.xlabel("x1 axis")
plt.ylabel("function value")

In [ ]:
a.visualise2d(lim=5, n=100)
plt.plot([0, 0], [-5, 5],  c='grey', lw=1, ls='-.')
plt.plot([-5, 5], [0, 0],  c='grey', lw=1, ls='-.')
plt.plot([-5, 5], [-5, 5], c='grey', lw=1, ls='-.')
plt.plot([-5, 5], [5, -5],  c='grey', lw=1, ls='-.')

In [ ]:
a.visualise3d(lim=2, n=50)

In [ ]:
a.visulise_gradient(lim=2, n=50)

### bukin function
\begin{equation*}
f(x) = 100 \sqrt{\left|x_2 - 0.01x_1^2\right|} + 0.01\left|x_1 + 10\right| \\
f'(x) = \left(-x_1\frac{1}{\sqrt{\left|x_2 - 0.01x_1^2\right|}} sgn(x_2 - 0.01x_1^2) + 0.01 sgn(x_1+10), \;\;
\frac{50}{\sqrt{\left|x_2 - 0.01x_1^2\right|}} sgn(x_2 - 0.01x_1^2) \right) \\
f^*(x^*) = 0, x^*=(-10, 1)
\end{equation*}

In [ ]:
class bukin(objective_func):
    '''
    non-disappearing gradient
    large gradient and uncontinuous gradient around ridge/local optimal
    optimum: 0
    optimal: (-10, 1)
    '''
    def __init__(self):
        self.optimal = np.array([-10, 1])
        self.optimum = 0
        self.lim = 15
    def func(self, x):
        return 100 * np.sqrt(np.abs(x[1] - 0.01 * x[0]**2)) + 0.01 * np.abs(x[0] + 10)
    def dfunc(self, x):
        arg1 = x[1] - 0.01 * x[0]**2
        arg2 = 50 / np.sqrt(np.abs(arg1)) * np.sign(arg1) if arg1 != 0 else 0
        return np.array([- 0.02 * x[0] * arg2 + 0.01 * np.sign(x[0] + 10), arg2])
    def get_optimal(self):
        return self.optimal
    def get_optimum(self):
        return self.optimum
b = bukin()
print("check func value: expected:", b.get_optimum(), ", actual:", b.func(b.get_optimal()))
print("check gradient: expected:[0,0], actual:", b.dfunc(b.get_optimal()))

In [ ]:
b.visualise2d(lim=15, n=100)
plt.xlabel("x1 axis")
plt.ylabel("x2 axis")
plt.title("function value of bukin")

In [ ]:
b.visualise2d_section(-10, 'y')

In [ ]:
b.visualise2d_section(1, 'x')

In [ ]:
b.visualise2d_section(0, 'x')

In [ ]:
b.visualise3d(lim=15, n=30)

In [ ]:
b.visulise_gradient(lim=15, n=50)

In [ ]:
b.dfunc(np.array([100,1.1]))

### eggholder function

\begin{equation*}
f(x) = -\left( x_2 + 47 \right) \sin\left( \sqrt{\left|\frac{x_1}{2} + (x_2 + 47)\right|} \right) -
x_1\sin\left( \sqrt{\left|x_1 - \left(x_2 + 47\right)\right|} \right) + 959.6407, \\
|x_1|, |x_2| \leq 512 \\
f'(x) = \left(-\sin \left( \sqrt{\left| x'_2 \right|} \right) - \frac{1}{2}(x_2 + 47)g(x'_1) - x_1 g(x'_2)     ,\;\; -\sin \left( \sqrt{\left| x'_1 \right|} \right) - (x_2 + 47)g(x'_1) + x_1 g(x'_2)   \right)\\
g(x'_j) = \cos\left( \sqrt{\left|x'_j\right|} \right) \frac{1}{2\sqrt{\left|x'_j\right|}} sgn(x'_j)  \\
x'_1 = \frac{x_1}{2} + (x_2 + 47), \\
x'_2 = x_1 - (x_2 + 47)\\
f^*(x^*)=0, x^*=(512, 404.2319)
\end{equation*}





In [ ]:
class eggholder(objective_func):
    # evaluated domain: 
    def __init__(self):
        self.optimal = np.array([522, 413])
        self.optimum = 0
        self.lim = 550
    def func(self, x):
        if np.abs(x[0]) > self.lim or np.abs(x[1]) > self.lim:
            return 2e3
        arg1 = x[0]/2 + (x[1] + 47) 
        arg2 = x[0]   - (x[1] + 47)
        f = lambda xx: np.sin(np.sqrt(np.abs(xx)))
        return -(x[1] + 47) * f(arg1) - x[0] * f(arg2) + 976.873
    def dfunc(self, x):
        if np.abs(x[0]) > self.lim or np.abs(x[1]) > self.lim:
            return np.array([0, 0])
        arg1 = x[0]/2 + (x[1] + 47) 
        arg2 = x[0]   - (x[1] + 47)
        g = lambda xx: np.cos(np.sqrt(np.abs(xx)))/np.sqrt(np.abs(xx))/2*np.sign(xx)
        f1 = (x[1] + 47) * g(arg1)
        f2 = x[0] * g(arg2)
        return np.array([-f1/2 - np.sin(np.sqrt(np.abs(arg2))) - f2, \
                         -f1   - np.sin(np.sqrt(np.abs(arg1))) + f2])
    def get_optimal(self):
        return self.optimal
    def get_optimum(self):
        return self.optimum
e = eggholder()
e.get_optimal()
print("check func value: expected:", e.get_optimum(), ", actual:", e.func(e.get_optimal()))
print("check gradient: expected:[0,0], actual:", e.dfunc(e.get_optimal()))

In [ ]:
e.visualise2d(lim=550, n=100)

In [ ]:
e.visualise2d_section(e.get_optimal()[1], 'x')

In [ ]:
e.visualise2d_section(e.get_optimal()[0], 'y')

In [ ]:
e.visualise3d(lim=600, n=100)

In [ ]:
e.visulise_gradient(lim=550, n=50)

In [ ]:
e.visulise_gradient(lim=550, n=50)

### tuned Ackley function
\begin{equation*}
f(x) = -20 \exp\left(-\frac{1}{5} \sqrt{\frac{1}{d}\sum_{i=1}^d x_i^2}\right) - \frac{1}{10}\left(-\frac{1}{5} \sqrt{\frac{1}{d}\sum_{i=1}^d x_i^2}\right)^4  \exp\left( \frac{1}{2d}\sum_{i=1}^d \cos\left( \pi x_i\right)\right) + 20\\
f'(x) = \left( g\left(x_1\right), \;\; g\left(x_2\right)\right), \\
g(x_j) = 4\frac{x_j}{d\sqrt{ \frac{1}{d} \sum_{i=1}^{d} x_i^2 } } \exp \left( -\frac{1}{5}\sqrt{ \frac{1}{d} \sum_{i=1}^{d} x_i^2 }\right) \\+ \frac{\pi}{20d}\left(-\frac{1}{5} \sqrt{\frac{1}{d}\sum_{i=1}^d x_i^2}\right)^4 \sin(\pi x_j) \exp\left( \frac{1}{2d} \sum_{i=1}^{d} \cos\left(\pi x_i \right) \right)\\-\frac{4}{6250d^2}\exp\left( \frac{1}{2d} \sum_{i=1}^{d} \cos\left(\pi x_i \right) \right) \left(\sum_{i=1}^{d} x_i^2\right) x_j \\
f^*(x^*) = e, x^*=(0,0)
\end{equation*}


In [ ]:
class tuned_ackley(objective_func):
    # evaluated domain: circle with radius 19
    def __init__(self, lim=22, dim=2):
        self.optimum = 0
        self.lim = lim
        self.dim = dim
        self.optimal = np.zeros((self.dim, ))
    def func(self, x):
        '''
        the period of local minimum along each axis is 1, integer coordinate (1,1), (2,3)... 
        x and y is interchangeable
        global minimum is 0 with arguments x=y=0
        symmetric along x=0, y=0, y=x lines
        disappearing global gradient when far away from optimal
        '''
        if np.linalg.norm(x) > self.lim:
            return 5e1
        arg1 = -0.2 * np.sqrt(np.power(x, 2).mean())
        arg2 = 0.5 * np.cos(np.pi*x).mean()
        return -20. * np.exp(arg1) - 0.1 * arg1**4 * np.exp(arg2) + 20.
    def dfunc(self, x):
        if np.linalg.norm(x) == 0:
            return x
        elif np.linalg.norm(x) > self.lim:
            return np.zeros((self.dim, ))
        arg1 = -0.2 * np.sqrt(np.power(x, 2).mean())
        arg2 = 0.5 * np.cos(np.pi*x).mean()
        g = lambda xx: -0.8 * xx / arg1 * np.exp(arg1) / self.dim + np.pi/20 * arg1**4 * np.sin(np.pi * xx) * np.exp(arg2) / self.dim \
                         - 4 * xx/6250 * np.exp(arg2) * np.power(x, 2).sum() / self.dim**2
        return g(x)
    def get_optimal(self):
        return self.optimal
    def get_optimum(self):
        return self.optimum
    def visualise2d_section(self, pos, dire):
        super().visualise2d_section(pos, dire)
        '''   plt.plot([-25, 25], [15.67, 15.67], label='y=15.67')
        plt.plot([-25, 25], [3.63, 3.66], label='y=3.66')
        plt.plot([12.96, 12.96], [0, 50], label='x=12.96')
        plt.plot([22, 22], [0, 50], label='x=22')
  '''
        plt.plot([22, 22], [3.26, 25], c='#1f77b4')
        plt.plot([-22, -22], [3.26, 25], c='#1f77b4')
        
at = tuned_ackley()
print("check func value at optimal is", at.get_optimum(), "(optimuam): ", at.func(at.get_optimal()))
print("check gradient is (0,0): ",at.dfunc(at.get_optimal()))

In [ ]:
at.func(np.array([22,0]))

In [ ]:
at.func(np.zeros(10,))

In [ ]:
at.visualise2d_section(0, 'x')
plt.title("tuned ackley, f(x1,0) cross section")
plt.xlabel("x1 axis")
plt.ylabel("function value")

In [ ]:
at.visualize2d_section_gradient(0, 'x')

In [ ]:
at.visualise3d(lim=25, n=150)

In [ ]:
at.visualise2d(lim=25, n=150)

In [ ]:
at.visulise_gradient(lim=25, n=51)